In [1]:
import torch
print(torch.cuda.is_available())  # should be True
print(torch.cuda.get_device_name(0))  # GPU name

True
NVIDIA GeForce RTX 3090


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_parquet("hf://datasets/Pedrampedram/CancerGov_QA/data/train-00000-of-00001-95d10b894a8afa4d.parquet")
df

,document_id,focus,qa_pair_id,question_id,question_type,question_text,answer_text
0,0000013_2,Chronic Eosinophilic Leukemia,1,0000013_2-1,information,What is (are) Chronic Eosinophilic Leukemia ?,Key Points\n - Chronic eosi...
1,0000013_2,Chronic Eosinophilic Leukemia,2,0000013_2-2,symptoms,What are the symptoms of Chronic Eosinophilic ...,Signs and symptoms of chronic eosinophilic leu...
2,0000013_2,Chronic Eosinophilic Leukemia,3,0000013_2-3,treatment,What are the treatments for Chronic Eosinophil...,Treatment of chronic eosinophilic leukemia may...
3,0000003_5,Childhood Soft Tissue Sarcoma,1,0000003_5-1,information,What is (are) Childhood Soft Tissue Sarcoma ?,Key Points\n\t\t\t\t\t\t\t ...
4,0000003_5,Childhood Soft Tissue Sarcoma,2,0000003_5-2,susceptibility,Who is at risk for Childhood Soft Tissue Sarco...,Having certain diseases and inherited disorder...
...,...,...,...,...,...,...,...
724,0000014_1,Endometrial Cancer,4,0000014_1-4,exams and tests,How to diagnose Endometrial Cancer ?,Tests that examine the endometrium are used to...
725,0000014_1,Endometrial Cancer,5,0000014_1-5,outlook,What is the outlook for Endometrial Cancer ?,Certain factors affect prognosis (chance of re...
726,0000014_1,Endometrial Cancer,6,0000014_1-6,stages,What are the stages of Endometrial Cancer ?,Key Points\n - After endome...
727,0000014_1,Endometrial Cancer,7,0000014_1-7,treatment,What are the treatments for Endometrial Cancer ?,Key Points\n - There are di...


In [4]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"Shape of training set: {train_df.shape}")
print(f"Shape of validation set: {validation_df.shape}")
print(f"Shape of test set: {test_df.shape}")

Shape of training set: (583, 7)
Shape of validation set: (73, 7)
Shape of test set: (73, 7)


In [5]:
"""
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
from tqdm import tqdm

# Load model + tokenizer
model_id = "openai/gpt-oss-20b"
#model_id = "openai/gpt-oss-6.9b"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32  
)

# Assume test_df has a column "prompt"
prompts = test_df["prompt"].tolist()

outputs = []
batch_size = 2   # adjust depending on your GPU memory

for i in tqdm(range(0, len(prompts), batch_size)):
    batch_prompts = prompts[i:i+batch_size]
    inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True, max_length=1024)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    out_tokens = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    
    decoded_batch = tokenizer.batch_decode(out_tokens, skip_special_tokens=True)
    outputs.extend(decoded_batch)

# Add results to DataFrame
test_df["model_output"] = outputs
test_df.to_csv("test_with_outputs.csv", index=False)

print("✅ Finished inference. Results saved to test_with_outputs.csv")
"""

'\nfrom transformers import AutoTokenizer, AutoModelForCausalLM\nimport torch\nimport pandas as pd\nfrom tqdm import tqdm\n\n# Load model + tokenizer\nmodel_id = "openai/gpt-oss-20b"\n#model_id = "openai/gpt-oss-6.9b"\n\ntokenizer = AutoTokenizer.from_pretrained(model_id)\nmodel = AutoModelForCausalLM.from_pretrained(\n    model_id,\n    torch_dtype=torch.float32  \n)\n\n# Assume test_df has a column "prompt"\nprompts = test_df["prompt"].tolist()\n\noutputs = []\nbatch_size = 2   # adjust depending on your GPU memory\n\nfor i in tqdm(range(0, len(prompts), batch_size)):\n    batch_prompts = prompts[i:i+batch_size]\n    inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True, max_length=1024)\n    inputs = {k: v.to(model.device) for k, v in inputs.items()}\n    \n    out_tokens = model.generate(\n        **inputs,\n        max_new_tokens=256,\n        temperature=0.7,\n        do_sample=True,\n        pad_token_id=tokenizer.eos_token_id,\n    )\n    \n    de

In [6]:
"""
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
from tqdm import tqdm

# Load model + tokenizer
model_id = "openai/gpt-oss-20b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,   # use float32 if CPU only
)

# Assume test_df has a column "prompt"
prompts = test_df["prompt"].tolist()

outputs = []
batch_size = 2   # adjust depending on your GPU memory

for i in tqdm(range(0, len(prompts), batch_size)):
    batch_prompts = prompts[i:i+batch_size]
    inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True, max_length=1024)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    out_tokens = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    
    decoded_batch = tokenizer.batch_decode(out_tokens, skip_special_tokens=True)
    outputs.extend(decoded_batch)

# Add results to DataFrame
test_df["model_output"] = outputs
test_df.to_csv("test_with_outputs.csv", index=False)

print("✅ Finished inference. Results saved to test_with_outputs.csv")
"""

'\nfrom transformers import AutoTokenizer, AutoModelForCausalLM\nimport torch\nimport pandas as pd\nfrom tqdm import tqdm\n\n# Load model + tokenizer\nmodel_id = "openai/gpt-oss-20b"\ntokenizer = AutoTokenizer.from_pretrained(model_id)\nmodel = AutoModelForCausalLM.from_pretrained(\n    model_id,\n    torch_dtype=torch.float32,   # use float32 if CPU only\n)\n\n# Assume test_df has a column "prompt"\nprompts = test_df["prompt"].tolist()\n\noutputs = []\nbatch_size = 2   # adjust depending on your GPU memory\n\nfor i in tqdm(range(0, len(prompts), batch_size)):\n    batch_prompts = prompts[i:i+batch_size]\n    inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True, max_length=1024)\n    inputs = {k: v.to(model.device) for k, v in inputs.items()}\n    \n    out_tokens = model.generate(\n        **inputs,\n        max_new_tokens=256,\n        temperature=0.7,\n        do_sample=True,\n        pad_token_id=tokenizer.eos_token_id,\n    )\n    \n    decoded_batc

In [7]:
"""
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import pandas as pd
from tqdm import tqdm

# Load your test dataset
# test_df = pd.read_csv("your_test_file.csv")  # make sure test_df has a column "prompt"

# Model ID
model_id = "EleutherAI/gpt-j-6B"

# Initialize tokenizer and model pipeline
generator = pipeline(
    "text-generation",
    model=model_id,
    device=-1  # -1 means CPU
)

prompts = test_df["input"].tolist()
outputs = []

# Generate outputs for each prompt
for prompt in tqdm(prompts):
    result = generator(
        prompt,
        max_length=256,   # adjust as needed
        do_sample=True,
        temperature=0.7
    )
    outputs.append(result[0]['generated_text'])

# Save results to DataFrame
test_df["model_output"] = outputs
test_df.to_csv("test_with_outputs.csv", index=False)
print("✅ Finished inference, results saved to test_with_outputs.csv")
"""

'\nfrom transformers import AutoTokenizer, AutoModelForCausalLM, pipeline\nimport pandas as pd\nfrom tqdm import tqdm\n\n# Load your test dataset\n# test_df = pd.read_csv("your_test_file.csv")  # make sure test_df has a column "prompt"\n\n# Model ID\nmodel_id = "EleutherAI/gpt-j-6B"\n\n# Initialize tokenizer and model pipeline\ngenerator = pipeline(\n    "text-generation",\n    model=model_id,\n    device=-1  # -1 means CPU\n)\n\nprompts = test_df["input"].tolist()\noutputs = []\n\n# Generate outputs for each prompt\nfor prompt in tqdm(prompts):\n    result = generator(\n        prompt,\n        max_length=256,   # adjust as needed\n        do_sample=True,\n        temperature=0.7\n    )\n    outputs.append(result[0][\'generated_text\'])\n\n# Save results to DataFrame\ntest_df["model_output"] = outputs\ntest_df.to_csv("test_with_outputs.csv", index=False)\nprint("✅ Finished inference, results saved to test_with_outputs.csv")\n'

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import pandas as pd
from tqdm import tqdm
import torch

model_id = "Qwen/Qwen2-0.5B"

# Load tokenizer + model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # FP16 for GPU
    device_map="auto"           # let accelerate handle GPU placement
)

# Do NOT set device here — the model already knows where to go
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

# Batched inference
prompts = test_df["question_text"].tolist()
outputs = []
batch_size = 8

for i in tqdm(range(0, len(prompts), batch_size)):
    batch_prompts = prompts[i:i+batch_size]
    results = generator(
        batch_prompts,
        max_new_tokens=128,
        truncation=True,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # results is a list of lists, one list per prompt
    for r in results:
        outputs.append(r[0]["generated_text"])

# Save outputs
test_df["model_output"] = outputs
test_df.to_csv("test_with_qwen_outputs_cancer_gov.csv", index=False)
print("✅ Finished inference on GPU using accelerate. Results saved.")


Device set to use cuda:0
100%|███████████████████████████████████████████| 10/10 [02:37<00:00, 15.77s/it]

✅ Finished inference on GPU using accelerate. Results saved.


In [11]:
"""
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

test_df = pd.read_csv("test_with_qwen_outputs_cancer_gov.csv")

references = [[ref.split()] for ref in test_df["answer_text"].tolist()]  # each ref must be a list of tokens
hypotheses = [hyp.split() for hyp in test_df["model_output"].tolist()]

# Smoothing function helps with short sentences
smooth = SmoothingFunction().method1

# Compute corpus BLEU
bleu_score = corpus_bleu(references, hypotheses, smoothing_function=smooth)
print(f"BLEU score: {bleu_score*100:.2f}")
"""

BLEU score: 0.04


NameError: name 'val_dataset' is not defined